# IMPORT

In [1]:
# pip install chromedriver_autoinstaller
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()

# pip install selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from dotenv import load_dotenv
import time

from datetime import datetime
import re
import locale

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup

import os
load_dotenv()

False

# SELENIUM - Booking Scrapping

In [2]:
url = "https://www.booking.com/hotel/ad/hotel-exe-prisma.es.html#tab-reviews"

In [3]:
times = 300

In [8]:
def get_html_reviews(url, times):
    driver = webdriver.Chrome()
    driver.get(url)

    # Click on "reject all"
    button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "onetrust-reject-all-handler")))
    button.click()

    # Scroll
    start = time.time()
    scrolling_time = 2

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        end = time.time()
        difference = end - start

        if difference > scrolling_time:
            break

    html_list = [] 

    # Click on "next page" arrow X times
    for _ in range(times):
        try:
            element = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CLASS_NAME, "pagenext")))
            element.click()
            time.sleep(1)

            # Retrieve HTML and add it to the list
            html = driver.execute_script("return document.body.outerHTML;")
            html_list.append(html)
        except:
            print("Failed to click 'Next' button.")
            time.sleep(2)  # (wait before retrying)

    driver.quit()

    return html_list


In [10]:
html_list = get_html_reviews(url, times)

Failed to click 'Next' button.
Failed to click 'Next' button.
Failed to click 'Next' button.
Failed to click 'Next' button.
Failed to click 'Next' button.
Failed to click 'Next' button.
Failed to click 'Next' button.
Failed to click 'Next' button.
Failed to click 'Next' button.


# BS4 - Extracting info from reviews

In [11]:
def extract_info_from_post (post):
    # client
    client = post.find("span", {"class": "bui-avatar-block__title"}).text
    # title post
    title = post.find("h3", {"class": "c-review-block__title c-review__title--ltr"}).text.strip()
    
     # review
    reviews = post.find_all("span", {"class": "c-review__body"})

    try:
        pos_review = reviews[0].text.strip()
    except IndexError:
        pos_review = np.nan
    try:
        neg_review = reviews[1].text.strip()
    except IndexError:
        neg_review = np.nan
    
    # date review
    dates = post.find_all("span", {"class": "c-review-block__date"})
    date_review = dates[1].text.strip()

    if date_review.startswith('Comentó en:'):
        date_review = date_review.split(':')[1].strip()
        day, month, year = date_review.split(' de ')
    else:
        day, month, year = date_review.split(' de ')

    month_number = {
        'enero': '01',
        'febrero': '02',
        'marzo': '03',
        'abril': '04',
        'mayo': '05',
        'junio': '06',
        'julio': '07',
        'agosto': '08',
        'septiembre': '09',
        'octubre': '10',
        'noviembre': '11',
        'diciembre': '12'
    }

    date_review = f"{day}/{month_number[month]}/{year}"


    # nationality
    nationality = post.find("span", {"class": "bui-avatar-block__subtitle"}).text.strip()
    # score
    score = post.find("div", {"class": "bui-review-score__badge"}).text.strip()
    # booking details
    info = post.find_all("div", {"class": "bui-list__body"})
    travel_type = info[2::3]
    travel_type = travel_type[0].text.strip() if travel_type else ""
    travel_type

    room_type = info[0::3]
    room_type = room_type[0].text.strip() if room_type else ""
    room_type

    # create dict
    new_dict = {"client": client, 
                "title": title, 
                "pos_review": pos_review,
                "neg_review": neg_review,
                "nationality": nationality,
                "score": score,
                "travel_type": travel_type,
                "room_type": room_type,
                "date_review": date_review}
     

    return new_dict

# DF

In [16]:
soup_list = []

# Create Bs4 for each HTML
for html in html_list:
    soup = BeautifulSoup(html, "html.parser")
    soup_list.append(soup)

# Extract info
new_list = []
for soup in soup_list:
    feed = soup.find("div", {"id": "review_list_page_container"})
    posts = feed.find_all("li", {"class": "review_list_new_item_block"})

    for post in posts:
        try:
            result = extract_info_from_post(post)
            if result not in new_list:
                new_list.append(result)
        except:
            pass
        
df = pd.DataFrame(new_list)
df.head(10)

,client,title,pos_review,neg_review,nationality,score,travel_type,room_type,date_review
0,Sandra,Excepcional,Restaurante cena,NaN,España,10,En pareja,Habitación Familiar Deluxe,27/04/2023
1,Augustbou,Muy bien,"Ubicación, aparcamiento, amabilidad del person...","No es un hotel con magníficas instalaciones, e...",España,"8,0",En pareja,Habitación Doble Deluxe,26/04/2023
2,Jeep,Muy bien,Tamaño de habitacion,Wi-fi colapsada,España,"8,0",En pareja,Habitación Doble Deluxe,24/04/2023
3,Carles,Fantástico,Todo,A la ducha de nuestra habitación le faltaba pr...,España,"9,0",En familia,Habitación Cuádruple Deluxe,23/04/2023
4,Miguel,Excepcional,Nos gustó todo en general fabuloso,Ok,España,10,En familia,Habitación Cuádruple Deluxe,22/04/2023
5,Sara,Muy bien,Ubicación,Nada,España,"8,0",En grupo,Habitación Cuádruple Estándar,22/04/2023
6,Miguel,Muy bien,En el centro y parking,Hacía mucho calor y no pudimos bajar la calefa...,España,"8,0",En familia,Habitación Cuádruple Estándar,11/04/2023
7,Jose,RELAX,"BUENOS DIAS ,TENER APARCAMIENTO EN EL HOTEL","NO ES QUE NO ME GUSTARA ,PERO ESPERABA MAS DEL...",España,"8,0",En familia,Habitación Cuádruple Estándar,11/04/2023
8,Carla,Para repetir,El trato del personal fue muy agradable y la h...,Nada malo.,España,10,En pareja,Habitación Doble - 1 o 2 camas,11/04/2023
9,Alicia,Un gran hotel,El hotel está muy ubicado. Es elegante y moder...,"Nada, todo es genial",España,10,En pareja,Habitación Doble - 1 o 2 camas,10/04/2023


In [13]:
#df.to_csv('data/scrapping_booking_prisma.csv')